In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")

In [2]:
## Pdf reader
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('faq.pdf')
docs=loader.load()

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)
documents[:5]

[Document(page_content='Q: What is stpl\nA: A Short Term Personal Loan (STPL), also known as a Dial for Cash facility, is an exclusive\nprogram under which you can get a pre-approved loan directly into your savings bank account. It\nallows repayment in easy EMIs (Equated Monthly Installments). Some benefits of STPL include:\n1. Pre-approved limit: You will receive the pre-approved amount, subject to eligibility criteria and\napproval from the lending institution.\n2. No documentation required: The process is relatively straightforward compared to traditional loan\napplications that typically require extensive paperwork.\n3. Paperless money remittance: Funds are transferred directly into your bank account without the\nneed for physical cash transfer or processing, making it convenient and fast.\n4. Nominal interest rate and processing fees: STPL often offers lower interest rates compared to\nother loan products, with minimal or reduced processing fees.', metadata={'source': 'faq.pdf', '

In [5]:
#Vector Store Conversion
from langchain_community.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings(model="llama3")
from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(documents,embeddings)


In [6]:
db

In [7]:
query="What is STPL"
result=db.similarity_search(query)
result[0].page_content

'A: You will receive the short term personal loan (STPL) money directly into your savings bank\naccount. This is part of an exclusive program that allows you to access a pre-approved loan amount,\neven over and above your credit limit, with easy EMIs. The process typically does not require\nextensive documentation, offers paperless remittance, and includes nominal interest rates and\nprocessing fees. To apply for the STPL, you can follow the provided link or contact the relevant\ninstitution directly.'

In [8]:
## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. Take a Deep breath and 
work on this problem step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>
Question: {input}""")

In [10]:
from langchain_community.llms import Ollama
llm=Ollama(model="llama3")

In [11]:
## Chain Introduction
## Create Stuff Docment Chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)

In [12]:
"""
Retrievers: A retriever is an interface that returns documents given
 an unstructured query. It is more general than a vector store.
 A retriever does not need to be able to store documents, only to 
 return (or retrieve) them. Vector stores can be used as the backbone
 of a retriever, but there are other types of retrievers as well. 
 https://python.langchain.com/docs/modules/data_connection/retrievers/   
"""

retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x10511d690>)

In [13]:
"""
Retrieval chain:This chain takes in a user inquiry, which is then
passed to the retriever to fetch relevant documents. Those documents 
(and original inputs) are then passed to an LLM to generate a response
https://python.langchain.com/docs/modules/chains/
"""
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [16]:
response=retrieval_chain.invoke({"input":"How to apply stpl"})

In [17]:
response['answer']

"I'm going to take a deep breath and provide a detailed answer step by step based on the provided context.\n\nTo apply for a Short Term Personal Loan (STPL), also known as Dial for Cash, follow these steps:\n\n**Step 1: Check eligibility and pre-approved limit**\nEnsure that you are eligible for this exclusive program under which STPL is offered by your bank. This might depend on factors such as credit score and existing relationship with the bank.\n\n**Step 2: Obtain no documentation required (if necessary)**\nSince there's no requirement for extensive paperwork to apply for an STPL, it is advisable to gather any necessary documents that can support your application, if requested by the bank. These may include proof of income or employment.\n\n**Step 3: Visit a nearby branch or dial the bank's helpline number**\nTo initiate the process and get a pre-approved loan amount directly into your savings account, contact your bank through their physical branches or by calling their STPL helpl